Задание:

С помощью метода get_prepared_all_vacancies_from_hh
загрузить 10000 вакансии с сайта hh.ru.

Договоритесь чтобы вакансии не повторялись.

Сохранить результат в виде csv файла с тремя столбцами:
1. id вакансии
2. сырой json
3. подготовленное описание вакансии

Не забывайте про обработку ошибок

In [71]:
!pip install langdetect

In [2]:
!pip install aiohttp

Ибо Колаб кривой, добавляем немного [магии](https://gist.github.com/korakot/f1d835321fa14075227add61b90debb3) для обработки асинхронных запросов

In [3]:
!pip install nest-asyncio

In [4]:
import nest_asyncio
nest_asyncio.apply()

Тестовый запуск асинхронного кода

In [5]:
import asyncio
import time

In [6]:
async def waiter() -> None:
    await cook('Паста', 2)
    await cook('Салат Цезарь', 3)
    await cook('Отбивные', 1)

async def cook(order, time_to_prepare):
    print(f'Новый заказ: {order}')
    await asyncio.sleep(time_to_prepare)
    print(order, '- готово')

asyncio.run(waiter())

Новый заказ: Паста
Паста - готово
Новый заказ: Салат Цезарь
Салат Цезарь - готово
Новый заказ: Отбивные
Отбивные - готово


Импорты библиотек для работы с данными

In [74]:
from langdetect import detect

In [7]:
import pandas as pd
import json

In [8]:
import asyncio
import aiohttp

# Загрузка данных

In [17]:
import spacy

In [18]:
import pandas as pd
import numpy as np
import re

In [19]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer 
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ko\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ko\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ko\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Предобработка данных

## Загрузка данных

In [97]:
name = '../data/vacancy.csv'
df = pd.read_csv(name, sep=",")
df

,vacancy_id,json,description,tokens
0,33551858.0,"{'id': '33551858', 'premium': False, 'billing_...",<p> </p> <p>Приглашаем на серьёзную ответствен...,приглаша серьезн ответствен работ приветств ли...
1,33551859.0,"{'id': '33551859', 'premium': False, 'billing_...",<p><strong><em>Департамент управления барами –...,департамент управлен бар – « вкусн » подраздел...
2,33551860.0,"{'id': '33551860', 'premium': False, 'billing_...",<p>«Сеть фитнес клубов ФизКульт является призн...,« сет фитнес клуб физкульт явля призна лидер ф...
3,33551861.0,"{'id': '33551861', 'premium': False, 'billing_...",<p><strong>Должностные обязанности:</strong></...,должностн обязан • осуществлен расчетно-кассов...
4,33551862.0,"{'id': '33551862', 'premium': False, 'billing_...",<p><strong>Обязанности:</strong></p> <ul> <li>...,обязан амбулаторно-поликлиническ требован нали...
...,...,...,...,...
9833,33587460.0,"{'id': '33587460', 'premium': False, 'billing_...",<p>Рудник «Ирокинда» расположен в Муйском райо...,рудник « ирокинд » располож муйск район респуб...
9834,33587461.0,"{'id': '33587461', 'premium': False, 'billing_...",<p>Рудник «Ирокинда» расположен в Муйском райо...,рудник « ирокинд » располож муйск район респуб...
9835,33587462.0,"{'id': '33587462', 'premium': False, 'billing_...",<strong>Обязанности:</strong> <ul> <li>знание ...,обязан знан программ 1с 8 3 зарплат кадр оформ...
9836,33587463.0,"{'id': '33587463', 'premium': False, 'billing_...",<p><strong>LaGarderobe</strong> – это мультибр...,lagarderob – эт мультибрендов магазин элегантн...


# Предобработка данных

## Обработка данных

РЕАЛИЗОВАТЬ `remove_garbage`

Используйте регулярные выражения:

- IN: str '<p\>ТРЕБОВАНИЯ: <br /\> - обязательно высшее или незаконченное высшее экономическое образование;...';
- OUT: str 'ТРЕБОВАНИЯ: - обязательно высшее или незаконченное высшее экономическое образование;...'

In [21]:
def remove_garbage(raw_text: str):
    text = re.sub(r"<[^>]*>", "", raw_text)
    return text

In [63]:
remove_garbage("<p\>ТРЕБОВАНИЯ: <br /\>")

'ТРЕБОВАНИЯ: '

РЕАЛИЗОВАТЬ `tokenize`

Используйте `word_tokenize` из `nltk.tokenize`

- IN: str 'ТРЕБОВАНИЯ: - обязательно высшее или незаконченное высшее экономическое образование;...'
- OUT: list ['требования', ':', '-', 'обязательно', 'высшее', 'или', 'незаконченное', 'высшее', 'экономическое', 'образование']

In [22]:
def tokenize(raw_text):
    tokens = word_tokenize(raw_text)
    return tokens

In [23]:
tokenize("ТРЕБОВАНИЯ: - обязательно высшее или незаконченное высшее экономическое образование;...")

['ТРЕБОВАНИЯ',
 ':',
 '-',
 'обязательно',
 'высшее',
 'или',
 'незаконченное',
 'высшее',
 'экономическое',
 'образование',
 ';',
 '...']

РЕАЛИЗОВАТЬ `to_base_form`

Можно использовать и лемматизацию и стемминг: pymystem3, ntlk, Spacy, gensim

- IN: str 'Красивая мама красиво мыла раму'
- OUT: str 'красивый мама красиво мыть рама'

In [24]:
# spacy.load('en_core_web_trf')
# spacy.load('ru_core_news_lg')

In [57]:
def to_base_form(raw_text, is_lemming=True):
    if type(raw_text) == str:
        if is_lemming:
            # Лемматизация
            lemmatizer = spacy.load('ru_core_news_lg', disable=['parser', 'ner'])
            tokens = lemmatizer(raw_text)
            token = " ".join([token.lemma_ for token in tokens])
            
            lemmatizer = spacy.load('en_core_web_trf', disable=['parser', 'ner'])
            tokens = lemmatizer(token)
            token = " ".join([token.lemma_ for token in tokens])
        else:
            # Стемминг
            stemmer = SnowballStemmer("russian")
            tokens = tokenize(raw_text)
            stemming = [stemmer.stem(word) for word in tokens]
            token = " ".join(stemming)
            
            stemmer = SnowballStemmer("english")
            tokens = tokenize(token)
            stemming = [stemmer.stem(word) for word in tokens]
            token = " ".join(stemming)
    
    if type(raw_text) in [list, np.ndarray]:
        # Стемминг
        stemmer = SnowballStemmer("russian")
        token = [stemmer.stem(word) for word in raw_text]
    return token

In [64]:
str_test = "<p>ТРЕБОВАНИЯ: <br /> - обязательно высшее или незаконченное высшее экономическое образование;..."
print(tokenize(to_base_form(remove_garbage(str_test))))

str_test = "We are going to fire this world"
tokenize(to_base_form(remove_garbage(str_test)))

D:\ProgramData\Anaconda3\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


['требования', ':', '-', 'обязательно', 'высокий', 'или', 'незаконченный', 'высокий', 'экономический', 'образование', ';', '...']


['we', 'be', 'go', 'to', 'fire', 'this', 'world']

In [65]:
str_test = "<p>ТРЕБОВАНИЯ: <br /> - обязательно высшее или незаконченное высшее экономическое образование;...НИЯ: <br /> - обязательно высшее или незаконченное высшее экономическое образование;...НИЯ: <br /> - обязательно высшее или незаконченное высшее экономическое образование;...НИЯ: <br /> - обязательно высшее или незаконченное высшее экономическое образование;...НИЯ: <br /> - обязательно высшее или незаконченное высшее экономическое образование;...НИЯ: <br /> - обязательно высшее или незаконченное высшее экономическое образование;...НИЯ: <br /> - обязательно высшее или незаконченное высшее экономическое образование;...НИЯ: <br /> - обязательно высшее или незаконченное высшее экономическое образование;...НИЯ: <br /> - обязательно высшее или незаконченное высшее экономическое образование;...НИЯ: <br /> - обязательно высшее или незаконченное высшее экономическое образование;...НИЯ: <br /> - обязательно высшее или незаконченное высшее экономическое образование;...НИЯ: <br /> - обязательно высшее или незаконченное высшее экономическое образование;...НИЯ: <br /> - обязательно высшее или незаконченное высшее экономическое образование;...НИЯ: <br /> - обязательно высшее или незаконченное высшее экономическое образование;...НИЯ: <br /> - обязательно высшее или незаконченное высшее экономическое образование;...НИЯ: <br /> - обязательно высшее или незаконченное высшее экономическое образование;...НИЯ: <br /> - обязательно высшее или незаконченное высшее экономическое образование;...НИЯ: <br /> - обязательно высшее или незаконченное высшее экономическое образование;...НИЯ: <br /> - обязательно высшее или незаконченное высшее экономическое образование;...НИЯ: <br /> - обязательно высшее или незаконченное высшее экономическое образование;...НИЯ: <br /> - обязательно высшее или незаконченное высшее экономическое образование;...НИЯ: <br /> - обязательно высшее или незаконченное высшее экономическое образование;...НИЯ: <br /> - обязательно высшее или незаконченное высшее экономическое образование;...НИЯ: <br /> - обязательно высшее или незаконченное высшее экономическое образование;...НИЯ: <br /> - обязательно высшее или незаконченное высшее экономическое образование;...НИЯ: <br /> - обязательно высшее или незаконченное высшее экономическое образование;...НИЯ: <br /> - обязательно высшее или незаконченное высшее экономическое образование;...НИЯ: <br /> - обязательно высшее или незаконченное высшее экономическое образование;...НИЯ: <br /> - обязательно высшее или незаконченное высшее экономическое образование;..."
print(tokenize(to_base_form(remove_garbage(str_test), is_lemming=False)))

['требован', ':', '-', 'обязательн', 'высш', 'ил', 'незакончен', 'высш', 'экономическ', 'образован', ';', '...', 'ни', ':', '-', 'обязательн', 'высш', 'ил', 'незакончен', 'высш', 'экономическ', 'образован', ';', '...', 'ни', ':', '-', 'обязательн', 'высш', 'ил', 'незакончен', 'высш', 'экономическ', 'образован', ';', '...', 'ни', ':', '-', 'обязательн', 'высш', 'ил', 'незакончен', 'высш', 'экономическ', 'образован', ';', '...', 'ни', ':', '-', 'обязательн', 'высш', 'ил', 'незакончен', 'высш', 'экономическ', 'образован', ';', '...', 'ни', ':', '-', 'обязательн', 'высш', 'ил', 'незакончен', 'высш', 'экономическ', 'образован', ';', '...', 'ни', ':', '-', 'обязательн', 'высш', 'ил', 'незакончен', 'высш', 'экономическ', 'образован', ';', '...', 'ни', ':', '-', 'обязательн', 'высш', 'ил', 'незакончен', 'высш', 'экономическ', 'образован', ';', '...', 'ни', ':', '-', 'обязательн', 'высш', 'ил', 'незакончен', 'высш', 'экономическ', 'образован', ';', '...', 'ни', ':', '-', 'обязательн', 'высш', '

РЕАЛИЗОВАТЬ `remove_stop_words`

Используйте stopwords из nltk.corpus 

- IN: list ['требования', ':', '-', 'обязательно', 'высший', 'или', 'незаконченный', 'высший', 'экономический', 'образование']
- OUT: list ['требование', 'обязательно', 'высший', 'незаконченный', 'высший', 'экономический', 'образование']

In [37]:
import string

In [38]:
def remove_stop_words(raw_text):
    text_without_ru_stop = list(filter(lambda word: word not in stopwords.words('russian'), raw_text))
    text_without_ru_en_stop = list(filter(lambda word: word not in stopwords.words('english'), text_without_ru_stop))
    text = list(filter(lambda word: word not in string.punctuation, text_without_ru_en_stop))
    return text

In [66]:
print(remove_stop_words(tokenize("требования : - обязательно высший или незаконченный высший экономический образование")))
print(remove_stop_words(tokenize("Hello me and / or my brothers!")))

['требования', 'обязательно', 'высший', 'незаконченный', 'высший', 'экономический', 'образование']
['Hello', 'brothers']


РЕАЛИЗОВАТЬ `get_prepared_vacancy_description_from_hh`

Используйте реализованные ранее методы

- IN: int 72323
- OUT1: str vacancy_json dump
- OUT2: list[list] [['требование', 'обязательно', 'высший', 'незаконченный', 'высший', 'экономический', 'образование', ...], [...], ...]

In [40]:
def get_description_from_string(json_str):
    regexp = r'\'description\': \'([^\']*)\''
    match = re.search(regexp, str(json_str))
    return match.group(1) if match else ""

In [41]:
get_description_from_string("{'id': '72323', 'premium': False, 'billing_type': {'id': 'standard', 'name': 'Стандарт'}, 'relations': [], 'name': 'Консультант в отдел продаж 1С', 'insider_interview': None, 'response_letter_required': True, 'area': {'id': '2', 'name': 'Санкт-Петербург', 'url': 'https://api.hh.ru/areas/2'}, 'salary': None, 'type': {'id': 'open', 'name': 'Открытая'}, 'address': None, 'allow_messages': True, 'site': {'id': 'hh', 'name': 'hh.ru'}, 'experience': {'id': 'noExperience', 'name': 'Нет опыта'}, 'schedule': {'id': 'fullDay', 'name': 'Полный день'}, 'employment': {'id': 'full', 'name': 'Полная занятость'}, 'department': None, 'contacts': None, 'description': '<p>ТРЕБОВАНИЯ: <br />- обязательно высшее или незаконченное высшее экономическое образование; <br />- уверенный пользователь ПК; <br />- приветствуется знание программ 1С на уровне пользователя и/или основ бухгалтерского учета; <br />- готовность заниматься продвижением программных продуктов 1С. </p><p>ОБЯЗАННОСТИ: <br />Выезд к заказчику на демонстрацию программного обеспечения,<br />осуществление продаж, поддержка клиентской базы. <br />Поиск клиентов в обязанности не входит. <br />Офисно-разъездной характер работы, возможен гибкий график, неполный рабочий день (но не менее 3,5-4 дней в неделю). </p><p>УСЛОВИЯ: <br />Начальное обучение и стажировка, <br />зарплата – оклад от $ 200 + %, перспективы роста. </p><p>Высылая резюме, ОБЯЗАТЕЛЬНО укажите название вакансии в теме письма. </p>', 'branded_description': None, 'vacancy_constructor_template': None, 'key_skills': [], 'accept_handicapped': False, 'accept_kids': False, 'archived': True, 'response_url': None, 'specializations': [{'id': '2.43', 'name': 'Бухгалтер', 'profarea_id': '2', 'profarea_name': 'Бухгалтерия, управленческий учет, финансы предприятия'}, {'id': '2.179', 'name': 'Начальный уровень, Мало опыта', 'profarea_id': '2', 'profarea_name': 'Бухгалтерия, управленческий учет, финансы предприятия'}, {'id': '1.225', 'name': 'Продажи', 'profarea_id': '1', 'profarea_name': 'Информационные технологии, интернет, телеком'}], 'professional_roles': [], 'code': None, 'hidden': False, 'quick_responses_allowed': False, 'driver_license_types': [], 'accept_incomplete_resumes': False, 'employer': {'id': '3177', 'name': 'Первый Бит', 'url': 'https://api.hh.ru/employers/3177', 'alternate_url': 'https://hh.ru/employer/3177', 'logo_urls': {'original': 'https://hhcdn.ru/employer-logo-original/635336.png', '90': 'https://hhcdn.ru/employer-logo/2982496.png', '240': 'https://hhcdn.ru/employer-logo/2982497.png'}, 'vacancies_url': 'https://api.hh.ru/vacancies?employer_id=3177', 'trusted': True}, 'published_at': '2004-10-27T11:42:35+0400', 'created_at': '2004-10-27T11:42:35+0400', 'negotiations_url': None, 'suitable_resumes_url': None, 'apply_alternate_url': 'https://hh.ru/applicant/vacancy_response?vacancyId=72323', 'has_test': False, 'test': None, 'alternate_url': 'https://hh.ru/vacancy/72323', 'working_days': [], 'working_time_intervals': [], 'working_time_modes': [], 'accept_temporary': False}")

'<p>ТРЕБОВАНИЯ: <br />- обязательно высшее или незаконченное высшее экономическое образование; <br />- уверенный пользователь ПК; <br />- приветствуется знание программ 1С на уровне пользователя и/или основ бухгалтерского учета; <br />- готовность заниматься продвижением программных продуктов 1С. </p><p>ОБЯЗАННОСТИ: <br />Выезд к заказчику на демонстрацию программного обеспечения,<br />осуществление продаж, поддержка клиентской базы. <br />Поиск клиентов в обязанности не входит. <br />Офисно-разъездной характер работы, возможен гибкий график, неполный рабочий день (но не менее 3,5-4 дней в неделю). </p><p>УСЛОВИЯ: <br />Начальное обучение и стажировка, <br />зарплата – оклад от $ 200 + %, перспективы роста. </p><p>Высылая резюме, ОБЯЗАТЕЛЬНО укажите название вакансии в теме письма. </p>'

In [42]:
def get_prepared_vacancy_description(vacancy_description, is_lemming=True):
    vacancy_description_sentences = vacancy_description.split(".") # получить лист предложений из вакансии
    prepared_vacancy_description = []
    for sentence in vacancy_description_sentences:
        sentense_no_garbage = remove_garbage(sentence)
        tokens = tokenize(to_base_form(sentense_no_garbage, is_lemming=is_lemming))
        prepared_vacancy_description += remove_stop_words(tokens)
    return prepared_vacancy_description

In [43]:
def get_prepared_vacancy_from_base_string(vacancy_json, is_lemming=True):
    vacancy_description = get_description_from_string(vacancy_json) # получить описание вакансии из json
    prepared_vacancy_description = get_prepared_vacancy_description(vacancy_description, is_lemming=is_lemming)
    return vacancy_json, prepared_vacancy_description

In [44]:
def get_prepared_vacancy_from_hh(vacancy_id, is_lemming=True):
    base_url = 'https://api.hh.ru/vacancies/'
    vacancy_json = download_json(base_url + str(vacancy_id))
    str_json = asyncio.run(asyncio.gather(vacancy_json))[0]
    return get_prepared_vacancy_from_base_string(str_json, is_lemming=is_lemming)

In [45]:
get_prepared_vacancy_from_hh(72323, is_lemming=False)

("{'id': '72323', 'premium': False, 'billing_type': {'id': 'standard', 'name': 'Стандарт'}, 'relations': [], 'name': 'Консультант в отдел продаж 1С', 'insider_interview': None, 'response_letter_required': True, 'area': {'id': '2', 'name': 'Санкт-Петербург', 'url': 'https://api.hh.ru/areas/2'}, 'salary': None, 'type': {'id': 'open', 'name': 'Открытая'}, 'address': None, 'allow_messages': True, 'site': {'id': 'hh', 'name': 'hh.ru'}, 'experience': {'id': 'noExperience', 'name': 'Нет опыта'}, 'schedule': {'id': 'fullDay', 'name': 'Полный день'}, 'employment': {'id': 'full', 'name': 'Полная занятость'}, 'department': None, 'contacts': None, 'description': '<p>ТРЕБОВАНИЯ: <br />- обязательно высшее или незаконченное высшее экономическое образование; <br />- уверенный пользователь ПК; <br />- приветствуется знание программ 1С на уровне пользователя и/или основ бухгалтерского учета; <br />- готовность заниматься продвижением программных продуктов 1С. </p><p>ОБЯЗАННОСТИ: <br />Выезд к заказчику

РЕАЛИЗОВАТЬ `get_prepared_all_vacancies_from_hh`

Используйте `get_prepared_vacancy_from_hh` и `ThreadPoolExecutor` из `concurrent.futures`

- IN: list [int]
- OUT: list [vacancy_id, raw_description, prepared_description]

In [46]:
def get_prepared_all_vacancies_from_hh(vacancy_ids, is_lemming=True):
    return [get_prepared_vacancy_from_hh(vacancy_id, is_lemming=is_lemming) for vacancy_id in vacancy_ids]

In [48]:
get_prepared_all_vacancies_from_hh(range(72323, 72326), is_lemming=False)

[("{'id': '72323', 'premium': False, 'billing_type': {'id': 'standard', 'name': 'Стандарт'}, 'relations': [], 'name': 'Консультант в отдел продаж 1С', 'insider_interview': None, 'response_letter_required': True, 'area': {'id': '2', 'name': 'Санкт-Петербург', 'url': 'https://api.hh.ru/areas/2'}, 'salary': None, 'type': {'id': 'open', 'name': 'Открытая'}, 'address': None, 'allow_messages': True, 'site': {'id': 'hh', 'name': 'hh.ru'}, 'experience': {'id': 'noExperience', 'name': 'Нет опыта'}, 'schedule': {'id': 'fullDay', 'name': 'Полный день'}, 'employment': {'id': 'full', 'name': 'Полная занятость'}, 'department': None, 'contacts': None, 'description': '<p>ТРЕБОВАНИЯ: <br />- обязательно высшее или незаконченное высшее экономическое образование; <br />- уверенный пользователь ПК; <br />- приветствуется знание программ 1С на уровне пользователя и/или основ бухгалтерского учета; <br />- готовность заниматься продвижением программных продуктов 1С. </p><p>ОБЯЗАННОСТИ: <br />Выезд к заказчик

## Формирование и обработка датасета

In [9]:
async def download_json(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            if response.status == 200:
                data_json = str(json.loads(await response.text()))
                return data_json
            else:
                return None

# пример: download_json('https://api.hh.ru/vacancies/43551857')

Получим объект типа `concurrent` - для дальнейшего вызова его в основном методе

In [10]:
def get_jsons_vacansions(n_vacancy, from_vacancy=1, path='https://api.hh.ru/vacancies/', verbose=False):
    # Download vacancies
    return [download_json(path + str(id)) for id in range(from_vacancy, from_vacancy + n_vacancy)]

Собственно, получение JSON-чиков по списку - стартует одновременно, получение асинхронно

In [100]:
def find_save_vacancies(df=None, n_vacancy=1, from_vacancy=1, jsons=[], name='vacancy.csv', verbose=False):
    if df is None:
        try:
            df = pd.read_csv(name)
        except FileNotFoundError:
            df = pd.DataFrame({
                'vacancy_id': [],
                'json': [],
                'description': [],
                'tokens': []
            })
    current_ids = df.vacancy_id
    if verbose:
        print('Length before download: {}'.format(df.shape[0]))
    paths_jsons = get_jsons_vacansions(n_vacancy, from_vacancy=from_vacancy)
    paths_jsons = list(filter(lambda x: x not in current_ids, paths_jsons))
    jsons = asyncio.run(asyncio.gather(*paths_jsons))
    # Убираем значения, где получен пустой json
    jsons = list(filter(lambda x: x is not None, jsons))
    # Получаем описание и токен
    descriptions = [get_description_from_string(s) for s in jsons]
    tokens = [' '.join(get_prepared_vacancy_description(description, is_lemming=False)) for description in descriptions]
    values = [{'vacancy_id': vacancy_id, 'json': json, 'description': description, 'tokens': token} 
              for vacancy_id, json, description, token in 
              zip(range(from_vacancy, from_vacancy + n_vacancy), jsons, descriptions, tokens)]
    # Оставляем только тексты с русским языком
    filtered_ru_values = list(filter(lambda x: detect(x['tokens']) == 'ru', values))
    df = df.append(filtered_ru_values, ignore_index=True)
    # Drop duplicates if exists (unnecessary)
    # df = df.drop_duplicates()
    if verbose:
        print('Length after download: {}'.format(df.shape[0]))
    df.to_csv(name, index=False)
    return df

Инициализируем датасет

In [101]:
df = None

Параметры получения JSON-чиков:
- `start_id` - id с которого начинаем получать вакансии
- `CUR_NUM` - по сути, сколько объектов уже получили по всем итерациям
- `NUM_TO_TAKE` - число объектов, которые получаем в данной операции

Особенности работы:
- Код выполняется [асинхронно](https://docs.python.org/3/library/asyncio.html)
- Код выполняется [параллельно](https://ru.stackoverflow.com/a/1302518/455418)
- Обрабатываются только ответы с кодом 200 (`OK`)
- За проход получает малое число объектов (около сотни), поэтому рекомендуется либо ставить малое число `NUM_TO_TAKE`, либо перезапускать несколько раз. За перезапуск отвечает цикл. Уже полученные строки не пытается получить повторно.

**ВНИМАНИЕ** общее число новых объектов может быть меньше `NUM_TO_TAKE` - в случае, если один или несколько ответов будут с кодом, отлиным от 200 (`OK`)

**ВНИМАНИЕ** `CUR_NUM` менять вручную

**ВНИМАНИЕ** До перезапуска датафрейм необходимо скачать, после перезапуска - загрузить в `Файлы`

ПЕРЕЗАПУСКАТЬ ТУТ

In [109]:
start_id = 33551857
CUR_NUM = 35600
NUM_TO_TAKE = 200
for i in range(3):
    df = find_save_vacancies(df=df, name=name, n_vacancy=NUM_TO_TAKE, from_vacancy=start_id + CUR_NUM)
    CUR_NUM += NUM_TO_TAKE
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10388 entries, 0 to 10387
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   vacancy_id   10388 non-null  float64
 1   json         10388 non-null  object 
 2   description  10388 non-null  object 
 3   tokens       10388 non-null  object 
dtypes: float64(1), object(3)
memory usage: 324.8+ KB


In [110]:
CUR_NUM

36200

Получим 10000 объектов датасета и сохраним их

In [119]:
df = df.reset_index(drop=True)
df = df.drop_duplicates()
df = df.reset_index(drop=True)
if df.shape[0] > 10000:
    df = df.drop(range(10000, df.shape[0]))

In [120]:
df

,vacancy_id,json,description,tokens
0,33551858.0,"{'id': '33551858', 'premium': False, 'billing_...",<p> </p> <p>Приглашаем на серьёзную ответствен...,приглаша серьезн ответствен работ приветств ли...
1,33551859.0,"{'id': '33551859', 'premium': False, 'billing_...",<p><strong><em>Департамент управления барами –...,департамент управлен бар – « вкусн » подраздел...
2,33551860.0,"{'id': '33551860', 'premium': False, 'billing_...",<p>«Сеть фитнес клубов ФизКульт является призн...,« сет фитнес клуб физкульт явля призна лидер ф...
3,33551861.0,"{'id': '33551861', 'premium': False, 'billing_...",<p><strong>Должностные обязанности:</strong></...,должностн обязан • осуществлен расчетно-кассов...
4,33551862.0,"{'id': '33551862', 'premium': False, 'billing_...",<p><strong>Обязанности:</strong></p> <ul> <li>...,обязан амбулаторно-поликлиническ требован нали...
...,...,...,...,...
9995,33587501.0,"{'id': '33587505', 'premium': False, 'billing_...",<p><em><strong>ИТАЛЬЯНСКАЯ ОБУВНАЯ ФАБРИКА МИЗ...,итальянск обувн фабрик мизр групп ищет рабоч о...
9996,33587502.0,"{'id': '33587506', 'premium': False, 'billing_...",<p>В ООО <strong>«Канц-Лайн»</strong> - компан...,оо « канц-лайн » компан снабжа предприят крым ...
9997,33587503.0,"{'id': '33587507', 'premium': False, 'billing_...",<p>В ООО <strong>«Канц-Лайн»</strong> - компан...,оо « канц-лайн » компан снабжа предприят крым ...
9998,33587504.0,"{'id': '33587508', 'premium': False, 'billing_...",<p>Сеть магазинов товаров для животных Мир Хво...,сет магазин товар животн мир хвостат интернет-...


Сохраним датасет

In [121]:
df.to_csv(name, sep=',', index=False)

## Обработка загруженного датасета

In [69]:
df

,vacancy_id,json,description,tokens
0,33551857.0,"{'id': '33551857', 'premium': False, 'billing_...",<strong>Обов&#39;язки:</strong> <ul> <li>Чітке...,39 язк чітке і якісн виконан своєї робот прове...
1,33551858.0,"{'id': '33551858', 'premium': False, 'billing_...",<p> </p> <p>Приглашаем на серьёзную ответствен...,приглаша серьезн ответствен работ приветств ли...
2,33551859.0,"{'id': '33551859', 'premium': False, 'billing_...",<p><strong><em>Департамент управления барами –...,департамент управлен бар – « вкусн » подраздел...
3,33551860.0,"{'id': '33551860', 'premium': False, 'billing_...",<p>«Сеть фитнес клубов ФизКульт является призн...,« сет фитнес клуб физкульт явля призна лидер ф...
4,33551861.0,"{'id': '33551861', 'premium': False, 'billing_...",<p><strong>Должностные обязанности:</strong></...,должностн обязан • осуществлен расчетно-кассов...
...,...,...,...,...
9995,33587460.0,"{'id': '33587460', 'premium': False, 'billing_...",<p>Рудник «Ирокинда» расположен в Муйском райо...,рудник « ирокинд » располож муйск район респуб...
9996,33587461.0,"{'id': '33587461', 'premium': False, 'billing_...",<p>Рудник «Ирокинда» расположен в Муйском райо...,рудник « ирокинд » располож муйск район респуб...
9997,33587462.0,"{'id': '33587462', 'premium': False, 'billing_...",<strong>Обязанности:</strong> <ul> <li>знание ...,обязан знан программ 1с 8 3 зарплат кадр оформ...
9998,33587463.0,"{'id': '33587463', 'premium': False, 'billing_...",<p><strong>LaGarderobe</strong> – это мультибр...,lagarderob – эт мультибрендов магазин элегантн...


Получим описания датасета

In [49]:
df.description = [get_description_from_string(s) for s in df.json]

получим токены датасета

In [60]:
df.tokens = [' '.join(get_prepared_vacancy_description(description, is_lemming=False)) for description in df.description]

In [61]:
df

,vacancy_id,json,description,tokens
0,33551857.0,"{'id': '33551857', 'premium': False, 'billing_...",<strong>Обов&#39;язки:</strong> <ul> <li>Чітке...,39 язк чітке і якісн виконан своєї робот прове...
1,33551858.0,"{'id': '33551858', 'premium': False, 'billing_...",<p> </p> <p>Приглашаем на серьёзную ответствен...,приглаша серьезн ответствен работ приветств ли...
2,33551859.0,"{'id': '33551859', 'premium': False, 'billing_...",<p><strong><em>Департамент управления барами –...,департамент управлен бар – « вкусн » подраздел...
3,33551860.0,"{'id': '33551860', 'premium': False, 'billing_...",<p>«Сеть фитнес клубов ФизКульт является призн...,« сет фитнес клуб физкульт явля призна лидер ф...
4,33551861.0,"{'id': '33551861', 'premium': False, 'billing_...",<p><strong>Должностные обязанности:</strong></...,должностн обязан • осуществлен расчетно-кассов...
...,...,...,...,...
9995,33587460.0,"{'id': '33587460', 'premium': False, 'billing_...",<p>Рудник «Ирокинда» расположен в Муйском райо...,рудник « ирокинд » располож муйск район респуб...
9996,33587461.0,"{'id': '33587461', 'premium': False, 'billing_...",<p>Рудник «Ирокинда» расположен в Муйском райо...,рудник « ирокинд » располож муйск район респуб...
9997,33587462.0,"{'id': '33587462', 'premium': False, 'billing_...",<strong>Обязанности:</strong> <ul> <li>знание ...,обязан знан программ 1с 8 3 зарплат кадр оформ...
9998,33587463.0,"{'id': '33587463', 'premium': False, 'billing_...",<p><strong>LaGarderobe</strong> – это мультибр...,lagarderob – эт мультибрендов магазин элегантн...


In [88]:
df.to_csv(name, sep=',', index=False)

# Вывод

Были проведены работы по составлению датасета